# Preamble

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os, sys
sys.path.append('../SatLib')
# import satPlotTools as spt
import satbox as sb
# import satbox_MJD as sb_MJD
import orbitalMechanics as om
import utils as utils
# import utils_MJD as utils_MJD
import CZMLExtractor_MJD as CZMLExtractor_MJD


In [6]:
import numpy as np
from poliastro import constants
from poliastro.earth import Orbit
from poliastro.earth.sensors import min_and_max_ground_range, ground_range_diff_at_azimuth
from poliastro.bodies import Earth
from poliastro.maneuver import Maneuver
from poliastro.twobody.propagation import propagate
from poliastro.twobody.propagation import cowell
from poliastro.core.perturbations import J2_perturbation
from poliastro.core.propagation import func_twobody
from poliastro.util import norm
import astropy
import astropy.units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import Angle

In [7]:
import matplotlib.pyplot as plt
from poliastro.plotting.static import StaticOrbitPlotter
from poliastro.plotting import OrbitPlotter3D, OrbitPlotter2D
%matplotlib notebook

# Test constellation propagation

## Create constellation object

In [9]:
# %debug
i = 45 * u.deg
t = 10
p = 2
f = 1
alt = 500 * u.km
walker = sb.Constellation.from_walker(i, t, p, f, alt)

/Users/manweichan/Desktop/MITPhD/SAS/code/.venv/lib/python3.7/site-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


### Create constellation sim object

In [41]:
t2propagate = 1 * u.day
tStep = 10 * u.s
walkerSim = sb.SimConstellation(walker, t2propagate, tStep, verbose = True)

### Propagate

In [42]:
# %debug
walkerSim.propagate()

### Plot 3D Initial Satellites

In [43]:
op = walkerSim.initConstellation.plot_sats([0,1,2,3])
op.show()

## Extract data from simulated constellation

In [44]:
# outputData = walkerSim.get_relative_velocity_analysis()

## Getting InterSatellite Links

In [45]:
import Interval_Finder as IF 


In [46]:
relative_position_data = IF.get_relative_position_data(walkerSim,2000)

In [47]:
satellites = IF.find_feasible_links(walkerSim, 10, relative_position_data)

In [48]:
L_avail = IF.get_multi_availability(walkerSim, satellites, relative_position_data)

In [49]:
L_poly = IF.get_multi_poly(walkerSim, satellites, relative_position_data)

In [50]:
"""lon = 40*u.deg 
lat = 70*u.deg 
h = 0*u.km
GS = sb_MJD.GroundLoc(lon,lat,h)"""

'lon = 40*u.deg \nlat = 70*u.deg \nh = 0*u.km\nGS = sb_MJD.GroundLoc(lon,lat,h)'

In [51]:
"""times = ['2000-01-01T00:00:00','2000-01-01T23:59:35']
t = Time(times)
print(GS.propagate_to_ECI(t))"""

"times = ['2000-01-01T00:00:00','2000-01-01T23:59:35']\nt = Time(times)\nprint(GS.propagate_to_ECI(t))"

In [52]:
L=[]
lon = 40*u.deg 
lat = 50*u.deg 
h = 0*u.km
GS0 = sb.GroundLoc(lon,lat,h)
L.append(GS0)
lon1 = 10*u.deg 
lat1 = 40*u.deg 
h1 = 0*u.km
GS1 = sb.GroundLoc(lon1,lat1,h1)
L.append(GS1)
lon2 = 40*u.deg 
lat2 = 70*u.deg 
h2 = 0*u.km
GS2 = sb.GroundLoc(lon2,lat2,h2)
L.append(GS2)
print(L)


[<satbox.GroundLoc object at 0x1565dea20>, <satbox.GroundLoc object at 0x1565de3c8>, <satbox.GroundLoc object at 0x1565de630>]


In [53]:
times = walkerSim.constellation.planes[0].sats[0].rvECEF.obstime.isot

In [54]:
relative_position_data_gs = IF.get_relative_position_data_gs(walkerSim,L,1000)


In [55]:
objects = IF.find_feasible_links_gs(walkerSim,10,3,relative_position_data_gs)

In [56]:
L_avail_gs = IF.get_multi_availability_gs(walkerSim, objects, relative_position_data_gs)

In [57]:
L_poly_gs = IF.get_multi_poly_gs(walkerSim, objects, relative_position_data_gs)

In [58]:
GS_pos =[[lon, lat], [lon1, lat1],[lon2, lat2]]

In [59]:
walker.generate_czml_file(fname='UROPTest',prop_duration=1,sample_points=100, satellites=satellites,objects=objects, L_avail=L_avail,L_poly=L_poly, L_avail_gs=L_avail_gs, L_poly_gs=L_poly_gs, GS_pos=GS_pos, GS=True, show_polyline=True)